<a href="https://colab.research.google.com/github/vickky0182/Personal/blob/main/TransformerFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi --query-gpu=memory.used --format=csv

memory.used [MiB]
0 MiB


In [ ]:
import torch
import pandas as pd
import torch.utils.data as data_utils
import numpy as np
import os
import random
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
file_path = "/content/drive/My Drive/Colab Notebooks/new_dataset/"

In [ ]:
fault_paths = [file_path + f"fault{i}/" for i in range(12)]

In [ ]:
def remove_zero(i,a):
  target_column = a
  min_value = i[target_column].min()
  max_value = i[target_column].max()
  random_values = np.random.uniform(min_value, max_value, len(i))
  i.loc[i[target_column] == 0, target_column] = random_values[i[target_column] == 0]
  return i

In [ ]:
def add_rows(a,c,n):
  lis = a.values.tolist()
  lower_limit = a[c].min()  # Lower limit of the range
  upper_limit = a[c].max()  # Upper limit of the range
  for i in range(n):
    random_int = random.randint(int(lower_limit), int(upper_limit))
    lis.append([random_int])
  new=pd.DataFrame(lis, columns=[c])
  return new
  # target_column = c
  # median_value = a[target_column].median()
  # a.loc[a[target_column] == 0, target_column] = median_value
  # return a

In [ ]:
def read_file(a,b,f):
  c=a+b+"/"

  column_names = ['wastage', 'Throttle_Pos_Area(%)', 'engine_speed', 'Pressure', 'Temp']
  i=c+b+"_inputdata.txt"
  input = pd.read_csv(i, delimiter='\s+', header=None, names=column_names)


  column_names = ['Engine_speed']
  o=c+b+"_omega_eREFdata.txt"
  omega = pd.read_csv(o, delimiter='\s+', header=None, names=column_names)

  column_names = ['Engine_Torque']
  t=c+b+"_Tq_eREFdata.txt"
  torque = pd.read_csv(t, delimiter='\s+', header=None, names=column_names)

  column_names = ["Tmp_AF","P_AF","Tmp_CMP","P_CMP","Tmp_IC","P_IC","Tmp_IN_Man","P_IN_Man","Tmp_EX_Man","P_Ex_Man","Tmp_Turb","P_Turb","Turb_Speed"]
  s=c+b+"_statedata.txt"
  state = pd.read_csv(s, delimiter='\s+', header=None, names=column_names)

  column_names = ['TMP_Cmp','P_Cp','TMP_IC','P_IC','TMP_IN_Man','P_IN_Man','AF_MassFlow','E_Torque','P_EX_Man']
  ou=c+b+"_outputdata.txt"
  output =pd.read_csv(ou, delimiter='\s+', header=None, names=column_names)

  input=remove_zero(input,'engine_speed')

  torque=add_rows(torque,'Engine_Torque',len(input)-len(torque))
  omega=add_rows(omega,'Engine_speed',len(input)-len(omega))

  df_input = pd.DataFrame(input.values, columns=['wastage', 'Throttle_Pos_Area(%)', 'engine_speed', 'Pressure', 'Temp'])
  df_state = pd.DataFrame(state.values, columns=["Tmp_AF","P_AF","Tmp_CMP","P_CMP","Tmp_IC","P_IC","Tmp_IN_Man","P_IN_Man","Tmp_EX_Man","P_Ex_Man","Tmp_Turb","P_Turb","Turb_Speed"])
  df_output = pd.DataFrame(output.values, columns=['TMP_Cmp','P_Cp','TMP_IC','P_IC','TMP_IN_Man','P_IN_Man','AF_MassFlow','E_Torque','P_EX_Man'])
  df_new_omega = pd.DataFrame(omega.values, columns=['Engine_speed'])
  df_new_torque = pd.DataFrame(torque.values, columns=['Engine_Torque'])
  df_fault = pd.DataFrame([f]*len(input), columns=['Fault'])

  df_input=df_input.drop('engine_speed', axis=1)
  df_output=df_output.drop('E_Torque', axis=1)

  merged_df = pd.concat([df_input, df_state, df_output, df_new_omega, df_new_torque,df_fault], axis=1)

  return merged_df

In [ ]:
combined_df = pd.DataFrame()

In [ ]:

for fault_path in fault_paths:

    fault = int(fault_path.split("fault")[-1].replace("/", ""))  # Extract the fault number from the path

    wltp_folders = [folder for folder in os.listdir(fault_path) if not folder.startswith('.')]  # Exclude files or folders starting with a dot

    for wltp_folder in wltp_folders:
        wltp_folder_path = fault_path + wltp_folder  # Path to the WLTP folder
        df = read_file(fault_path, wltp_folder, fault)  # Read the data from the WLTP folder
        combined_df = pd.concat([combined_df, df], axis=0)  # Append the data to the combined DataFrame
    print("Fault"+ str(fault) +" read Successfully")

Fault0 read Successfully
Fault1 read Successfully
Fault2 read Successfully
Fault3 read Successfully
Fault4 read Successfully
Fault5 read Successfully
Fault6 read Successfully
Fault7 read Successfully
Fault8 read Successfully
Fault9 read Successfully
Fault10 read Successfully
Fault11 read Successfully


In [ ]:
fault = combined_df['Fault']

In [ ]:
combined_df = combined_df.drop('Fault', axis=1)

In [ ]:
fault

0         0
1         0
2         0
3         0
4         0
         ..
47316    11
47317    11
47318    11
47319    11
47320    11
Name: Fault, Length: 6446294, dtype: int64

In [ ]:
merged_tensor = torch.tensor(combined_df.values)
fault_tensor = torch.tensor(fault.values,dtype=merged_tensor.dtype)

In [ ]:
merged_tensor.size()

torch.Size([6446294, 27])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import StepLR, ExponentialLR, CosineAnnealingLR
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
class EncoderDecoderTransformer(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, num_layers, num_heads, dropout):
        super(EncoderDecoderTransformer, self).__init__()

        self.transformer = nn.Transformer(d_model=input_dim, nhead=num_heads, num_encoder_layers=num_layers, num_decoder_layers=num_layers, dim_feedforward=hidden_dim, dropout=dropout)

        # Output layer
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, merged_tensor):
        transformed = self.transformer(merged_tensor, merged_tensor)
        output = self.fc(transformed)
        return output

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, merged_tensor, fault_tensor):
        self.merged_tensor = merged_tensor
        self.fault_tensor = fault_tensor

    def __len__(self):
        return self.merged_tensor.shape[0]

    def __getitem__(self, idx):
        merged_row = self.merged_tensor[idx]
        fault_label = self.fault_tensor[idx]
        return merged_row, fault_label

In [ ]:
def train(model, criterion, optimizer, train_loader):
    model.train()
    train_loss = 0.0
    correct = 0

    for merged_row, fault_label in train_loader:
        optimizer.zero_grad()
        merged_row = merged_row.to(device).float()  # Convert to float
        fault_label = fault_label.to(device).long()  # Convert to long

        output = model(merged_row)
        loss = criterion(output, fault_label)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        predicted = torch.argmax(output, dim=1)
        correct += (predicted == fault_label).sum().item()

    train_loss /= len(train_loader)
    accuracy = correct / len(train_loader.dataset)
    return train_loss, accuracy

In [ ]:
def evaluate(model, criterion, val_loader):
    model.eval()
    val_loss = 0.0
    correct = 0

    with torch.no_grad():
        for merged_row, fault_label in val_loader:
            merged_row = merged_row.to(device).float()  # Convert to float
            fault_label = fault_label.to(device).long()  # Convert to long

            output = model(merged_row)
            loss = criterion(output, fault_label)

            val_loss += loss.item()
            predicted = torch.argmax(output, dim=1)
            correct += (predicted == fault_label).sum().item()

    val_loss /= len(val_loader)
    accuracy = correct / len(val_loader.dataset)
    return val_loss, accuracy

In [ ]:
input_dim = 27
output_dim = 12 # Number of fault classes
hidden_dim = 64
num_layers = 2
num_heads = 9
dropout = 0.01
learning_rate = 0.0001
batch_size = 512
num_epochs = 10

In [ ]:
model = EncoderDecoderTransformer(input_dim, output_dim, hidden_dim, num_layers, num_heads, dropout).to(device)


In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create train and test datasets
merged_train, merged_test, fault_train, fault_test = train_test_split(
    merged_tensor, fault_tensor, test_size=0.3, random_state=42
)

# Create train and validation datasets
train_dataset = CustomDataset(merged_train, fault_train)
test_dataset = CustomDataset(merged_test, fault_test)

In [ ]:
# Create train and validation data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
loss = []
va_loss =[]
acc = []
v_acc = []

In [ ]:
for epoch in range(1, num_epochs + 1):
    train_loss, train_accuracy = train(model, criterion, optimizer, train_loader)
    val_loss, val_accuracy = evaluate(model, criterion, test_loader)
    loss.append(train_loss)
    va_loss.append(val_loss)
    acc.append(train_accuracy)
    v_acc.append(val_accuracy)
    print(f'Epoch {epoch}/{num_epochs}')
    print(f'Train Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.4f}')
    print(f'Val Loss: {val_loss:.4f} - Val Accuracy: {val_accuracy:.4f}')
    print('---------------------------------------------')

Epoch 1/10
Train Loss: 2.3033 - Train Accuracy: 0.1774
Val Loss: 2.1219 - Val Accuracy: 0.2431
---------------------------------------------
Epoch 2/10
Train Loss: 2.0474 - Train Accuracy: 0.2652
Val Loss: 1.9327 - Val Accuracy: 0.3004
---------------------------------------------
Epoch 3/10
Train Loss: 1.9427 - Train Accuracy: 0.2945
Val Loss: 1.8819 - Val Accuracy: 0.3112
---------------------------------------------
Epoch 4/10
Train Loss: 1.8949 - Train Accuracy: 0.3084
Val Loss: 1.8563 - Val Accuracy: 0.3213
---------------------------------------------
Epoch 5/10
Train Loss: 1.8675 - Train Accuracy: 0.3174
Val Loss: 1.8417 - Val Accuracy: 0.3264
---------------------------------------------
Epoch 6/10
Train Loss: 1.8473 - Train Accuracy: 0.3251
Val Loss: 1.8145 - Val Accuracy: 0.3373
---------------------------------------------


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plotting the Loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), loss, label='Train Loss')
plt.plot(range(1, num_epochs + 1), va_loss, label='Val Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
print()
print()
# Plotting the Accuracy
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), acc, label='Train Accuracy')
plt.plot(range(1, num_epochs + 1), v_acc, label='Val Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()